In [10]:
import pickle
import numpy as np
from scipy.stats import entropy
from tqdm import tqdm
from datasets import load_dataset

In [11]:
wikibio = load_dataset("wiki_bio", split="test")

In [1]:
from utils import get_selfcheck_data
data = get_selfcheck_data()

In [6]:
data[1]['gpt3_text']

"Matthew Aylmer, 1st Baron Aylmer (1708–1794) was an Irish soldier and colonial administrator. He was born in Dublin, the son of a barrister, and was educated at Trinity College, Dublin. He joined the British Army in 1727 and served in the War of the Austrian Succession and the Seven Years' War. He was promoted to lieutenant-general in 1772 and was appointed Governor of Gibraltar in 1775. He was made a baron in 1782 and was appointed Commander-in-Chief of the British forces in North America in 1783. He was recalled in 1790 and died in London in 1794. He was buried in Westminster Abbey."

In [12]:
wiki_id = data[1]['wiki_bio_test_idx']
concept = wikibio[wiki_id]["input_text"]["context"].strip()
concept

'matthew aylmer , 1st baron aylmer'

In [4]:
label_mapping = {
    'accurate': 0.0,
    'minor_inaccurate': 0.5,
    'major_inaccurate': 1.0,
}

In [5]:
human_label_detect_False   = {}
human_label_detect_False_h = {}
human_label_detect_True    = {}
for i_ in range(len(dataset)):
    dataset_i = dataset[i_]
    idx = dataset_i['wiki_bio_test_idx']
    raw_label = np.array([label_mapping[x] for x in dataset_i['annotation']])
    human_label_detect_False[idx] = (raw_label > 0.499).astype(np.int32).tolist()
    human_label_detect_True[idx]  = (raw_label < 0.499).astype(np.int32).tolist()
    average_score = np.mean(raw_label)
    if average_score < 0.99:
        human_label_detect_False_h[idx] = (raw_label > 0.99).astype(np.int32).tolist()

In [6]:
len(human_label_detect_False), len(human_label_detect_True), len(human_label_detect_False_h)

(238, 238, 206)

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [14]:
from sagemaker.predictor import retrieve_default
endpoint_name = "jumpstart-dft-deepseek-llm-r1-disti-20250504-191212"
predictor = retrieve_default(endpoint_name)
payload = {
    "messages": [
        {
            "role": "user",
            "content": "what is 10+1."
        }
    ],
    "max_tokens": 128
}
response = predictor.predict(payload)
print(response)
# payload = {
#     "messages": [
#         {
#             "role": "user",
#             "content": "What is 1+1?"
#         },
#         {
#             "role": "assistant",
#             "content": "It's 2."
#         },
#         {
#             "role": "user",
#             "content": "Explain more!"
#         }
#     ],
#     "max_tokens": 128
# }
# response = predictor.predict(payload)
# print(response)
# payload = {
#     "messages": [
#         {
#             "role": "user",
#             "content": "Create a Flappy Bird game in Python."
#         }
#     ],
#     "max_tokens": 128
# }
# response = predictor.predict(payload)
# print(response)

{'id': 'chatcmpl-139705941338704', 'object': 'chat.completion', 'created': 1746388447, 'choices': [{'index': 0, 'message': {'role': 'assistant', 'reasoning_content': 'First, I need to add the numbers 10 and 1.\n\nStarting from 10, I add 1 to it.\n\nCounting up, 10 plus 1 equals 11.\n\nTherefore, the result of 10 + 1 is 11.\n', 'content': '\n\n**Solution:**\n\nWe need to compute the sum of 10 and 1.\n\n1. **Count \\(10 + 1\\):**\n\n\\[\n10 + 1 = 11\n\\]\n\n2. **Final Answer:**\n\n\\[\n\\boxed{11}\n\\]\n\n<box>The result'}, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 13, 'completion_tokens': 128, 'total_tokens': 141}}


In [ ]:
from workflow import run_review_pipeline, run_detection_pipeline

# query = "What are the main differences between GPT-3.5 and GPT-4?"
# run_review_pipeline(query)

sentence = "Matthew Aylmer, 1st Baron Aylmer (1708–1794) was an Irish soldier and colonial administrator."
passage = "Matthew Aylmer, 1st Baron Aylmer (1708–1794) was an Irish soldier and colonial administrator. He was born in Dublin, the son of a barrister, and was educated at Trinity College, Dublin. He joined the British Army in 1727 and served in the War of the Austrian Succession and the Seven Years' War. He was promoted to lieutenant-general in 1772 and was appointed Governor of Gibraltar in 1775. He was made a baron in 1782 and was appointed Commander-in-Chief of the British forces in North America in 1783. He was recalled in 1790 and died in London in 1794. He was buried in Westminster Abbey."
run_detection_pipeline(sentence, passage)